In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import torch as torch
import os
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import pixiedust

Pixiedust database opened successfully


In [2]:
DATA_DIRECTORY = "../datos/"

In [3]:
diagnosticos  = pd.read_excel(DATA_DIRECTORY+"RESUMEN TAC CEREBRALES.xlsx")

In [4]:
# diccionario cuya llave es el id de paciente y el valor una lista 
# donde cada elemento de la lista es la matriz de una i
diccionario_imagenes_pacientes = dict()

for paciente in diagnosticos.paciente:
    directorio_paciente = DATA_DIRECTORY+"paciente_"+str(paciente)
    archivos_paciente = os.listdir(directorio_paciente)
    
    lista_imagenes_paciente = []
    for archivo in archivos_paciente:
        if archivo.endswith(".jpg"):
            imagen = mpimg.imread(directorio_paciente+"/"+archivo)
            lista_imagenes_paciente.append(imagen)
            
    diccionario_imagenes_pacientes[paciente] = lista_imagenes_paciente
    

## Modelos y arquitecturas
### Arquitecturas experimental  DNC
* Alimentamos al modelo imagen por imagen y se presenta un solo diagnostico por paciente
* El controller de la DNC esta compuesto por una convnet

In [5]:
CONTROLLER_OUTPUT_SIZE = 1024

In [6]:
#TODO: cambiar valores quemados por valores parametrizados y calculos dependientes
class ConvController(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1,4,kernel_size=3,stride=1)
        self.fc1  =  torch.nn.Linear(262144,CONTROLLER_OUTPUT_SIZE)
        
    def forward(self,x):
        h = self.conv1(x)
        
        #flatten
        h =  x.view(-1,x.shape[1]*x.shape[2]*x.shape[3])
        h =  self.fc1(h)
        
        return h #h_t in my txt

In [7]:

#TODO: cambiar valores quemados por valores parametrizados y calculos dependientes
#TODO: cordar por que en algun momento le puse bias = False a los pesos del vector de salida de la DNC


class DNC(torch.nn.Module):
    
    def __init__(self,controller,memory_size = (10,10),read_heads = 1):
        super().__init__()
        self.controller = controller
        self.N = memory_size[0] # number of memory locations
        self.W = memory_size[1] # word size of the memory 
        self.R = read_heads # number of read heads
        self.WS = 1 #not in the paper(they use 1), but used as a parametrizable number of write heads for further experiments
        self.interface_vector_size = (self.W*self.R) + (self.W*self.WS) + (2*self.W) + (5*self.R) + 3
        
        # inicialization st to random just for testing, remember to put on zeros
        self.memory_matrix = self.memory_matrix =  nn.Parameter(torch.zeros(size=memory_size),requires_grad= False) 
        
        #1024 es el tamaño del vector de salida del controlador, 1 es el tamaño de salida de la dnc
        self.output_vector_linear = torch.nn.Linear(CONTROLLER_OUTPUT_SIZE,1,bias=True) #W_y 
        self.interface_vector_linear = torch.nn.Linear(CONTROLLER_OUTPUT_SIZE,self.interface_vector_size,bias=False) #W_ξ
        self.read_vectors_to_output_linear = torch.nn.Linear(self.R*self.W,1,bias = False) #W_r in my txt
        
        self.read_keys = torch.Tensor(size=(self.R,self.W)).requires_grad_(False) # k_r in my txt
        self.read_strenghts = torch.Tensor(size=(self.R,1)).requires_grad_(False) #β_r
        
        self.read_weighting = torch.Tensor(torch.zeros(size=(self.R,self.N))).requires_grad_(False) #r_w
        
        self.write_key = torch.Tensor(size=(1,self.W)).requires_grad_(False) # k_w in my txt
        self.write_strenght = torch.Tensor(size=(1,1)).requires_grad_(False) # β_w
        
        self.write_weighting = torch.Tensor(torch.zeros(size=(1,self.N))).requires_grad_(False) # w_w
        
        self.usage_vector = torch.Tensor(torch.zeros(size=(1,self.N))).requires_grad_(False) #u_t
        
        self.memory_matrix_ones = torch.Tensor(torch.ones(size=memory_size)).requires_grad_(False) #E on paper
        
    def forward(self,x):
        print(self.memory_matrix)
        h_t = self.controller(x) #controller output called ht in the paper
        
        output_vector = self.output_vector_linear(h_t) # called Vt in the paper(υ=Wy[h1;...;hL]) v_o_t in my txt
        interface_vector = self.interface_vector_linear(h_t).data #called ξt(ksi) in the paper ,ξ_t in my txt
        
        self.read_keys.data = interface_vector[0,0:self.R*self.W].view((self.R,self.W)) #k_r in my txt
        
        #clamp temporary added because the exp was returning inf  values
        read_strenghts =  torch.clamp( interface_vector[0,self.R*self.W:self.R*self.W+self.R].view((self.R,1)),max=85)
        self.read_strenghts.data = self.oneplus(read_strenghts) #β_r
        
        self.write_key.data = interface_vector[0,self.R*self.W+self.R:self.R*self.W+self.R+self.W].view((1,self.W)) # k_w
        
        write_strenght = torch.clamp(interface_vector[:,self.R*self.W+self.R+self.W:self.R*self.W+self.R+self.W + 1].view((1,1)),max=85)
        self.write_strenght.data = self.oneplus(write_strenght) #β_w
        
        erase_vector = interface_vector[0,self.R*self.W+self.R+self.W + 1: self.R*self.W+self.R+self.W + 1 + self.W].view((1,self.W))
        erase_vector = torch.sigmoid(erase_vector) #e_t
        
        write_vector = interface_vector[0,self.R*self.W+self.R+self.W + 1 + self.W:self.R*self.W+self.R+self.W + 1 + 2*self.W].view((1,self.W)) #v_t
        
        free_gates  =  interface_vector[0,self.R*self.W+self.R+self.W + 1 + 2*self.W:self.R*self.W+2*self.R+self.W + 1 + 2*self.W].view((self.R,1)) #f_t
        free_gates =   torch.sigmoid(free_gates)
        
        allocation_gate = interface_vector[0,self.R*self.W+2*self.R+self.W + 1 + 2*self.W:self.R*self.W+2*self.R+self.W + 1 + 2*self.W+1]
        allocation_gate = torch.sigmoid(allocation_gate)
        
        write_gate = interface_vector[0,self.R*self.W+2*self.R+self.W + 1 + 2*self.W+1:self.R*self.W+2*self.R+self.W + 1 + 2*self.W+2]
        write_gate = torch.sigmoid( write_gate)
        
        
        # Escritura
        # TODO: verificar y/o experimentar si el ordern es :primero escribir y luego leer de la memoria(asi parece en el pazper)
        retention_vector = (1.0 - free_gates * self.read_weighting).prod(dim=0)
        self.usage_vector.data = (self.usage_vector +self.write_weighting - (self.usage_vector *self.write_weighting))*retention_vector #u_t
        allocation_weighting = self.calc_allocation_weighting(self.usage_vector)
        write_content_weighting = self.content_lookup(self.memory_matrix,self.write_key,self.write_strenght)

        self.write_weighting.data =  write_gate*(  
            (allocation_gate * allocation_weighting) +  ((1- allocation_gate)*write_content_weighting))
        
        new_memory_matrix = self.memory_matrix*(self.memory_matrix_ones - torch.matmul(self.write_weighting.t(),erase_vector)) + torch.matmul(self.write_weighting.t(),write_vector)
        
        self.memory_matrix.data = new_memory_matrix
        
        # read by content weithing(attention by similarity)
        read_content_weighting = self.content_lookup(self.memory_matrix,self.read_keys,self.read_strenghts)
        
        #read weithing is a combination of reading modes,TODO:add temporal attention not just by similarity
        self.read_weighting.data = read_content_weighting
        
        read_vectors = torch.matmul(self.read_weighting,self.memory_matrix).view((1,self.R*self.W)) #r in my txt
        read_heads_to_output = self.read_vectors_to_output_linear(read_vectors) #v_r_t in my t xt
        
        #TODO: experiment and decide if maintain sigmoid
        y_t = torch.sigmoid(output_vector + read_heads_to_output)
        return y_t
    
    def oneplus(self,x):
        # apply oneplus operation to a tensor to constrain it's elements to [1,inf)
        #TODO: check numerical statiliby as exp is returning inf for numbers like 710,emporary added clamp to 85
        return torch.log(1+torch.exp(x)) + 1
    
    def content_lookup(self,matrix,keys,strengths):
        # returns a probability distribution over the memory locations 
        # with higher probability to memory locations with bigger similarity to the keys
        # bigger strenght make more aggresive distributions ,for example a distribution (0.2,0.3,0.5) with
        # bigger strenght becomes (0.1,0.12,0.78)
        # returns tensor of shape (read keys,memory size) = (R,N)
        keys_norm =  torch.sqrt(torch.sum(keys**2,dim=1).unsqueeze(dim=1))
        matrix_norm = torch.sqrt(torch.sum(matrix**2,dim=1))
        norms_multiplication = keys_norm*matrix_norm
        # calc cosine similarity between keys and memory locations(1e-6 is used avoiding div by 0)
        divide_zero_prevent_factor = torch.zeros_like(norms_multiplication,requires_grad=False).add_(1e-6)
        cosine_similarity = torch.matmul(keys,matrix.t())/(torch.max(norms_multiplication,divide_zero_prevent_factor))
        
        # do a "strenght" softmax to calculate the probability distribution
        numerator = torch.exp(cosine_similarity*strengths)
        denominator = numerator.sum(dim=1).unsqueeze(dim=1)

        distribution = numerator/denominator
        
        return distribution
    
    def calc_allocation_weighting(self,usage_vector):
        #print("usage vector",usage_vector)
        _,free_list = torch.topk(-usage_vector,self.N,dim=1) #φt indices of memory locations ordered by usage
        #print("free list",free_list)
        free_list = free_list.view(-1)
        #print("reshaped free list",free_list)
        _,ordered_free_list =  torch.topk(-free_list,self.N)
        ordered_free_list = ordered_free_list.view(-1)
        #print("ordered free list",ordered_free_list)
        ordered_usage_vector = usage_vector[:,free_list]
        #print("ordered usage vector",ordered_usage_vector)
        ordered_usage_vector_cumulative_product = torch.ones(size=(1,self.N+1))
        #print(ordered_usage_vector_cumulative_product)
        #print("cumprod ",ordered_usage_vector.cumprod(dim=1))
        ordered_usage_vector_cumulative_product[0,1:] = ordered_usage_vector.cumprod(dim=1)
        #print(ordered_usage_vector_cumulative_product)
        
        allocation_weighting = (1 - usage_vector)*ordered_usage_vector_cumulative_product[0,ordered_free_list]
        
        return  allocation_weighting
    
    def reset(self):
        self.memory_matrix = self.memory_matrix =  nn.Parameter(torch.zeros(size=(self.N,self.W)),requires_grad= False) 
        self.read_weighting = torch.Tensor(torch.zeros(size=(self.R,self.N))).requires_grad_(False) #r_w
        self.write_weighting = torch.Tensor(torch.zeros(size=(1,self.N))).requires_grad_(False) # w_w

## Experimentos
* Experimentando con DNC alimentando una imagen a la vez en orden aleatorio con pacientes también en orden aleatorio

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
EPOCHS = 3

In [10]:
conv_controller = ConvController()
dnc_model = DNC(controller=conv_controller,memory_size = (5,5),read_heads=1).to(device)

In [11]:
base_criterion = torch.nn.BCELoss()
def loss_function(y,y_hat,last_flag):
    #print(y,y_hat,last_flag)
    #base_criterion = torch.nn.BCELoss()
    return torch.full_like(y,last_flag) * base_criterion(y,y_hat)

In [12]:
criterion = loss_function
optimizer = optim.Adam(dnc_model.parameters(),lr=0.001)

In [ ]:
total_accuracies  = []
for epoch in range(EPOCHS):
    epoch_predictions = []
    epoch_real_values = []
    # en cada epoch procesar los pacientes en orden aleatorio
    pacientes = np.random.choice(np.array(diagnosticos.paciente),size= len(diagnosticos.paciente),replace=False)
    
    conteo_pacientes = 0
    for paciente in pacientes:
        #TODO: remover esta validacion, solo puesta para probar una unica iteracion en compu lenta
        if conteo_pacientes >= 99999999:
            break
            
        dnc_model.reset()
        
        imagenes_paciente = diccionario_imagenes_pacientes.get(paciente)
        diagnostico_hemorragia_paciente = np.array(float(diagnosticos[diagnosticos.paciente==paciente].hemorragia))
        tensor_diagnostico_hemorragia_paciente = torch.Tensor(diagnostico_hemorragia_paciente).to(device)
        
        indices_imagenes_pacientes = np.arange(0,len(imagenes_paciente)-1,step=1)
        indices_aleatorios_imagenes = np.random.choice(indices_imagenes_pacientes,len(indices_imagenes_pacientes),replace=False)
        
        for indice in indices_aleatorios_imagenes:
            last_image =  int(indice  == indices_aleatorios_imagenes[-1])
            
            optimizer.zero_grad()
            
            imagen_paciente = imagenes_paciente[indice]
            
            if imagen_paciente.shape != (512,512):
                #TODO: tread different image sizes with reshaping, resizing(or other ideas)
                continue
                
            tensor_imagen_paciente =  torch.unsqueeze(
                torch.unsqueeze( torch.Tensor(imagen_paciente),dim=0),dim=1).to(device)
            
            #print("Alimentando paciente {} e imagen {} al modelo".format(paciente,indice),imagen_paciente.shape)
            
            diagnostico_hemorragia_aproximado = dnc_model(tensor_imagen_paciente)
            
            if last_image:
                y_hat = diagnostico_hemorragia_aproximado.data.numpy()[0][0]
                y_hat_hard = float(y_hat >= 0.5)
                epoch_predictions.append(y_hat_hard)
                epoch_real_values.append(float(diagnostico_hemorragia_paciente))
                
                #print("--Flag ultima imagen:{} diagnostico:{} valor real{}".format(last_image,y_hat,diagnostico_hemorragia_paciente))
            
            loss = criterion(diagnostico_hemorragia_aproximado,tensor_diagnostico_hemorragia_paciente,last_image)
            loss.backward()
            optimizer.step()

                
        conteo_pacientes += 1
            
    epoch_predictions = np.array(epoch_predictions)
    epoch_real_values = np.array(epoch_real_values)
    correct_predictions = epoch_predictions == epoch_real_values
    accuracy = np.average(correct_predictions)
    total_accuracies.append(accuracy)
    print("Epoch {}: accuracy {}".format(epoch,accuracy),epoch_predictions,epoch_real_values)

print(np.average(total_accuracies))

In [14]:
#TODO: averiguar por que salen 6 tensores de parametros si solo se han declarado 3(al momento de correr lap rueba)
train_parmams = list(dnc_model.named_parameters())

for train_param in train_parmams:
    print(train_param[0])

memory_matrix
controller.conv1.weight
controller.conv1.bias
controller.fc1.weight
controller.fc1.bias
output_vector_linear.weight
output_vector_linear.bias
interface_vector_linear.weight
read_vectors_to_output_linear.weight


In [15]:
dnc_model.memory_matrix.data


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

Meta (por detallar))
* L temporal link matrix

## Ideas
* Normalizar  el allocation weighitng con sofmax(en la primera iteración asigna todo el peso a la primera posición de memoria)
* Usar arquitectura similar a dueling network o inception para tener 2 caminos en las entradas.
* Cambiar el modelo original para leer antes que escribir y usar lo leido para sacar una predicción en ese punto en el tiempo(el modelo original lee de la memoria despues de escribir y usa la info leida en el siguiente paso)